# Mini Projekt - Baby Kyber

## Pierścień $\mathbb{Z}_{17}[X]/(X^4+1)$

In [13]:
# skopiuj pierścień ilorazowy wielomianów z pierwszych zajęć

import numpy as np
from math import ceil

class Zn():
    def __init__(self,num,N):
        self.N=N
        self.num=num%self.N
    def __str__(self):
        return str(self.num)
    def __add__(self,other):
        if isinstance(other,Zn):
            if self.N==other.N:
                return Zn(self.num+other.num,self.N)
            else:
                raise ValueError('Not compatible mod rings')
        elif isinstance(other,int):
            return Zn(self.num+other,self.N)
        else:
            raise TypeError(f"unsupported operand type(s) for +: 'Zn' and '{type(other).__name__}'")
    def __radd__(self,other):
        if isinstance(other,int):
            return Zn(self.num+other,self.N)
        else:
            raise TypeError(f"unsupported operand type(s) for +: '{type(other).__name__}' and 'Zn'")
    def __mul__(self,other):
        if isinstance(other,Zn):
            if self.N==other.N:
                return Zn(self.num*other.num,self.N)
            else:
                raise ValueError('Not compatible mod rings')
        elif isinstance(other,int):
            return Zn(self.num*other,self.N)
        else:
            raise TypeError(f"unsupported operand type(s) for *: 'Zn' and '{type(other).__name__}'")
    def __rmul__(self,other):
        if isinstance(other,int):
            return Zn(self.num*other,self.N)
        else:
            raise TypeError(f"unsupported operand type(s) for *: '{type(other).__name__}' and 'Zn'")
    def __sub__(self,other):
        if isinstance(other,Zn):
            if self.N==other.N:
                return Zn(self.num-other.num,self.N)
            else:
                raise ValueError('Not compatible mod rings')
        elif isinstance(other,int):
            return Zn(self.num-other,self.N)
        else:
            raise TypeError(f"unsupported operand type(s) for -: 'Zn' and '{type(other).__name__}'")
    def __rsub__(self,other):
        if isinstance(other,int):
            return Zn(other-self.num,self.N)
        else:
            raise TypeError(f"unsupported operand type(s) for -: '{type(other).__name__}' and 'Zn'")
    def __pow__(self,other):
        if isinstance(other,Zn):
            if self.N==other.N:
                return Zn(pow(self.num,other.num,self.N),self.N)
            else:
                raise ValueError('Not compatible mod rings')
        elif isinstance(other,int):
            return Zn(pow(self.num,other,self.N),self.N)
        else:
            raise TypeError(f"unsupported operand type(s) for **: 'Zn' and '{type(other).__name__}'")
    def __rpow__(self,other):
        if isinstance(other,int):
            return Zn(pow(other,self.num,self.N),self.N)
        else:
            raise TypeError(f"unsupported operand type(s) for **: '{type(other).__name__}' and 'Zn'")

    def __repr__(self):
        return str(self.num)


class Rq():
    q = 17
    f = np.array([1, 0, 0, 0, 1])
    def __init__(self, vec=None, random=False, distribution='uniform', size=4):
        if vec is None and random:
            if distribution == 'uniform':
                vec = np.random.randint(0, self.q, size)
            elif distribution == 'gaussian':
                vec = np.random.choice([-1, 0, 1], size, p=[1/5, 3/5, 1/5])

        if vec is not None:
            _, r = np.polydiv(np.array(vec), self.f)
            self.vec = r % self.q
            if len(self.vec) < size:
                self.vec = np.pad(self.vec, (0, size - len(self.vec)), 'constant')

    def __repr__(self):
        return str(self.vec.tolist())

    def __add__(self, other):
        return Rq(np.polyadd(self.vec, other.vec).astype(int), random=False)

    def __sub__(self, other):
        return Rq(np.polysub(self.vec, other.vec).astype(int), random=False)

    def __mul__(self, other):
        if isinstance(other, Rq):
            return Rq(np.polymul(self.vec, other.vec), random=False)
        elif isinstance(other, (int, np.integer)):
            scaled_vec = np.array(self.vec) * other
            _, r = np.polydiv(scaled_vec, self.f)
            return Rq(r % self.q, random=False)
        else:
            raise TypeError(f"Unsupported operand type(s) for *: 'Rq' and '{type(other).__name__}'")

    __rmul__ = __mul__


In [16]:
success = 0

## Baby Kyber

Zaimplementuj poniższe elementy kryptosystemu Baby Kyber tak, aby osiągnąć jak największą skuteczność w testach (przy niezerowych błędach). Wymagana minimalna skuteczność to 60%.

### Generowanie klucza

Zaimplementuj funkcję `key_gen()` realizującą generowanie klucza w kryptosystemie Baby Kyber. Funkcja ma zwracać `A,t,s`. Przetestuj, czy dla podanych $A,s,e$ otrzymasz poprawny wielomian $t$.

$A=\left[\begin{matrix}
    6x^3+16x^2+16x+11&9x^3+4x^2+6x+3\\
    5x^3+3x^2+10x+1&6x^3+x^2+9x+15
\end{matrix}\right]$

$\mathbf{s}=(-x^3-x^2+x,-x^3-x)$

$\mathbf{e}=(x^2,x^2-x)$

$\mathbf{t}=A\mathbf{s}+\mathbf{e}:\ \ \mathbf{t}=(16x^3+15x^2+7,10x^3+12x^2+11x+6)$

In [10]:
A = np.array([[Rq([6, 16, 16, 11]), Rq([9, 4, 6, 3 ])],[Rq([5, 3,  10, 1 ]), Rq([6, 1, 9, 15])],])
s = np.array([Rq([-1, -1, 1, 0]), Rq([-1, 0, -1, 0])])
e = np.array([Rq([1, 0, 0]), Rq([1, -1, 0])])
t = A @ s + e
t

array([[16. 15.  0.  7.], [10. 12. 11.  6.]], dtype=object)

In [4]:
def key_gen():
    matrix_size = (2, 2)
    A = np.array([[Rq(random=True, distribution='uniform') for _ in range(matrix_size[1])] for _ in range(matrix_size[0])])
    s = np.array([Rq(random=True, distribution='gaussian') for _ in range(matrix_size[1])])
    e = np.array([Rq(random=True, distribution='gaussian') for _ in range(matrix_size[1])])
    t = np.dot(A, s) + e
    return A, t, s


### Szyfrowanie

Zaimplementuj funkcję `encrypt(A,t,m)` realizującą szyfrowanie w kryptosystemie Baby Kyber a gdzie wejściowe `m` jest w postaci listy. Funkcja ma zwracać szyfrogram `c`. Przetestuj poprawność działania na poniższych danych.

$m=1\cdot x^3+0\cdot x^2+1\cdot x+1=x^3+x+1$

$\mathbf{r}=(-x^3+x^2,x^3+x^2-1)$

$\mathbf{e_1}=(x^2+x,x^2)$

$e_2=-x^3-x^2$

$\mathbf{u}=A^T\mathbf{r}+\mathbf{e_1}:\ \ \mathbf{u}=(11x^3+11x^2+10x+3,4x^3+4x^2+13x+11)$

$v=\mathbf{t}^T\mathbf{r}+e_2+\lfloor\frac{q}{2}\rceil m:\ \ v=8x^3+6x^2+9x+16$

$\mathbf{c}=(\mathbf{u},v):\ \ \mathbf{c}=((11x^3+11x^2+10x+3,4x^3+4x^2+13x+11),8x^3+6x^2+9x+16)$

In [11]:
m = Rq([1, 0, 1, 1])
r = np.array([Rq([-1, 1, 0, 0]), Rq([1, 1, 0, -1])])
e1 = np.array([Rq([1, 1, 0]), Rq([1, 0, 0])])
e2 = Rq([-1, -1, 0, 0])
u = A.T @ r + e1
v = t.T @ r + e2 + ceil(Rq.q / 2) * m
c = (u, v)
c


(array([[11. 11. 10.  3.], [ 4.  4. 13. 11.]], dtype=object),
 [ 8.  6.  9. 16.])

In [5]:
def encrypt(A, t, m):
    m = Rq([1, 0, 1, 1]) if isinstance(m, list) else m
    r = np.array([Rq(random=True, distribution='gaussian', size=4) for _ in range(2)])
    e1 = np.array([Rq(random=True, distribution='gaussian', size=3) for _ in range(2)])
    e2 = Rq(random=True, distribution='gaussian', size=4)
    u = A.T @ r + e1
    v = t.T @ r + e2 + ceil(Rq.q / 2) * m
    c = (u, v)
    return c

### Deszyfrowanie

Zaimplementuj funkcję `decrypt(c,s)` realizującą deszyfrowanie w kryptosystemie Baby Kyber. Funkcja ma zwracać ostateczną odszyfrowaną wiadomość `m_n`. Przetestuj działanie na poniższych danych.

$m_n=v-\mathbf{s}^T\mathbf{u}:\ \ m_n=8x^3+14x^2+8x+6$

$m_n=1\cdot x^3+0\cdot x^2+1\cdot x+1$


In [12]:
mn = v - s.T @ u
mn

[ 8. 14.  8.  6.]

In [6]:
import numpy as np

def decrypt(c, s):
    u, v = c
    mn = v - s.T @ u
    def round_coefficients(vec):
        rounded = []
        for coeff in vec.vec:
            if abs(coeff - 0) < abs(coeff - 9) or abs(coeff - 17) < abs(coeff - 9):
                rounded.append(0)
            else:
                rounded.append(1)
        return np.array(rounded)

    mn_rounded = round_coefficients(mn)
    mn_poly = Rq(mn_rounded, random=False)
    return mn_poly

### Testy

In [15]:
import secrets as sc

for i in range(1000):
    output=[]
    A,t,s=key_gen()
    m=[sc.choice((0,1)) for k in range(4)]
    c = encrypt(A,t,m)
    m_n = decrypt(c,s)
    if m_n == m:
        success += 1
print(f'Success rate: {success * 100 /1000} %')


Success rate: 63.0 %
